# Klassifikations-Challenge
In dieser Hausaufgabe sollen Sie das Wissen aus der Vorlesung und Übung einsetzen, um einen möglichst guten Klassifikator zu entwickeln.

In Warnemünde am Strand wurde ein Datensatz auf einem USB-Stick in einer Flasche angespült. Der Datensatz besteht aus 
256 Features, einer Sample ID und einem binären Target. Wir haben einen Trainings-Datensatz, und einen Test-Datensatz, bei dem die tatsächlichen Klassen zunächst nicht bekannt sind (möglicherweise wird zu einem geeigneten Zeitpunkt eine weitere Flasche angespült, die die Klassen für die Test-Daten enthält). 

Die Aufgabe besteht darin, einen Klassifikator zu entwickeln, der eine möglichst gute Accuracy auf den Test-Daten erreicht (das ist aktuell, ohne die Klassen der Test-Daten, natürlich noch schwer zu testen, aber uns fällt bestimmt etwas ein, wie wir die Accuracy mithilfe der Trainings-Daten abschätzen können). 

Ich habe mir den Datensatz schon einmal angeschaut, und einige Hinweise zusammengetragen:
* Bei der großen Anzahl von Features ist es natürlich schwer, eine Intuition für die Bedeutung der Features zu bekommen. Trotzdem könnte es lohnenswert sein, sich die Features einmal genau anzuschauen. Welchen Verteilungen folgen diese jeweils? Was ist der Wertebereich? Gibt es Features, die komplett anders aussehen als der Rest?
* Durch die 256 Features haben wir ein recht hochdimensionales Klassifikations-Problem. Vielleicht lohnt es sich, Methoden zur Dimensions-Reduktion oder zur Merkmals-Auswahl anzuwenden?
* Ich habe unten schon mal ein Template vorbereitet. Die Funktion train_classifier soll einen trainierten Klassifikator zurückgeben. Die Funktion predict soll ein numpy-array mit predictions zurückgeben, gegeben einen trainierten Klassifikator und Test-Daten. 
* Mein erster, relativ einfacher Versuch hat eine Accuracy von ungefähr 0.73 erreicht. Das Beste, was ich erreiche, ist eine Accuracy von 0.985.

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


train = pd.read_csv("train-data.csv")
test = pd.read_csv("test-data.csv")


In [7]:
train

,id,muggy.smalt.axolotl.pembus,dorky.peach.sheepdog.ordinal,slimy.seashell.cassowary.goose,snazzy.harlequin.chicken.distraction,frumpy.smalt.mau.ordinal,stealthy.beige.pinscher.golden,chummy.cream.tarantula.entropy,hazy.emerald.cuttlefish.unsorted,nerdy.indigo.wolfhound.sorted,...,wheezy.myrtle.mandrill.entropy,wiggy.lilac.lemming.sorted,gloppy.cerise.snail.contributor,woozy.silver.havanese.gaussian,jumpy.thistle.discus.sorted,muggy.turquoise.donkey.important,blurry.buff.hyena.entropy,bluesy.chocolate.kudu.fepid,gamy.white.monster.expert,target
0,27203b235016f125f33f6d4d48ca32e9,0.012506,1.371986,0.958408,1.289744,-0.446245,0.212161,1.321071,-1.538660,1.934172,...,-1.728378,-11.335665,-0.566079,0.761578,1.489018,-0.393349,-0.111531,-0.895503,-1.026561,1
1,fb0b3762745e1ad49592c20b580e4f5f,1.353671,-1.747058,-0.234683,-0.545985,-0.304538,-2.886683,2.409455,-0.049140,1.583701,...,-0.646548,2.818877,-0.551294,-0.051991,-0.032005,-0.913710,-1.151355,0.912585,1.176004,0
2,ec90499cf2cb93cd83b8f8218fcf708e,-0.565425,-0.887345,2.410747,-0.433751,-3.467221,0.115738,0.353592,0.657471,-4.870369,...,1.330056,-0.163911,0.735335,0.386948,0.030165,0.201055,0.418292,-0.718481,-0.141954,0
3,da8f0939447a819b1d841e4f0effc851,1.863101,-5.301145,1.078329,-0.433367,0.666427,0.162637,-0.200397,-0.687918,3.849059,...,-0.631285,0.705966,0.718329,-0.744975,-0.347844,0.424206,-0.544935,0.170643,-2.261862,0
4,3e0d6c0c4ff1e087fabfe06d9d4880f1,0.870740,0.301251,-0.260353,7.869271,0.215889,0.293024,-0.521552,-2.953138,0.280825,...,-0.341533,-0.157284,1.084306,0.372175,-0.364258,-4.451195,0.983008,0.025720,2.499734,0
5,39b9562f990690d8f0de030124f34ec9,0.707111,1.321919,-1.428193,-2.077623,0.099017,-0.338972,0.212824,-2.849258,-0.903054,...,-1.085079,0.272567,-0.903755,-0.008561,-0.250470,-0.406745,-3.922977,1.248686,-0.711253,0
6,a1c044b91862f48ca2cdf050b7756e2d,0.545070,0.623073,0.824878,1.071709,0.244311,0.268614,2.236471,-1.379155,0.767004,...,-3.611152,-0.928443,0.352006,-0.245199,-0.189771,-0.637101,0.418055,0.727722,0.099693,1
7,8bfb0c4e6a805c2daf61c00a56995802,-2.038132,-1.781465,0.410382,0.742281,0.099177,-0.046522,0.943223,-0.980203,0.256214,...,-1.473643,-0.005652,0.088238,-1.270416,1.028352,-1.387840,-2.123704,1.309572,0.045271,1
8,d33b53984a645d67c11cebefb47acaf4,-0.774551,-0.033070,-4.564802,0.807548,-0.042641,-0.077808,1.852714,0.207275,-0.168574,...,-0.975725,-2.982451,1.589711,0.091553,-0.442406,3.770906,-0.499079,-1.284714,-1.145274,0
9,741d61f055587a2f1adf47a496a76341,-0.036201,1.791346,0.247834,-0.497928,-1.249798,-1.282867,-1.117083,-0.807595,-1.037275,...,-0.557039,1.598398,0.344736,0.802997,-1.207934,1.718947,0.298804,-1.125477,0.316093,0


In [4]:
def train_classifier(train):
    #TODO
    return None
    
    
def predict(classifier,test):
    #TODO
    return None

def evaluate(prediction,truth):
    return np.mean(prediction == truth)

In [5]:
test2 = pd.read_csv("test-data-with-target.csv") #gibt es aktuell noch nicht...

classifier = train_classifier(train)
prediction = predict(classifier,test2)
print(evaluate(prediction,test2["target"]))

FileNotFoundError: [Errno 2] File b'test-data-with-target.csv' does not exist: b'test-data-with-target.csv'